# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data=pd.read_csv("weather_output.csv")
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Manono,-7.3000,27.4167,91.71,21.0,4.0,8.21,CD,1.627034e+09
1,Codrington,-38.2667,141.9667,46.83,85.0,99.0,12.35,AU,1.627034e+09
2,Jamestown,42.0970,-79.2353,61.72,92.0,82.0,1.59,US,1.627034e+09
3,Boma,7.0805,-2.1697,78.44,72.0,98.0,5.50,GH,1.627034e+09
4,Mataura,-46.1927,168.8643,46.47,94.0,42.0,3.24,NZ,1.627034e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

maxhumidity=city_data["Humidity"].max()

locations=city_data[["Lat","Lng"]]

weight=city_data["Humidity"]

In [4]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, \
    dissipating=False, max_intensity=maxhumidity, point_radius=3)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city=city_data[(city_data["Max Temp"]>70) & (city_data["Max Temp"]<80) \
    & (city_data["Wind Speed"]<10) & (city_data["Cloudiness"]==0)]

narrowed_city=narrowed_city.reset_index(drop=True)
narrowed_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kamenka,51.3223,42.7678,77.20,33.0,0.0,8.59,RU,1.627034e+09
1,Zlobin,45.2924,14.6541,77.16,30.0,0.0,4.61,HR,1.627035e+09
2,Mossamedes,-15.1961,12.1522,75.88,57.0,0.0,5.64,AO,1.627035e+09
3,Lüderitz,-26.6481,15.1594,70.50,20.0,0.0,9.24,NaN,1.627035e+09
4,Zarechnyy,53.2036,45.1923,72.48,37.0,0.0,8.57,RU,1.627035e+09
5,Henties Bay,-22.1160,14.2845,77.85,20.0,0.0,9.66,NaN,1.627035e+09
6,Médéa,36.2642,2.7539,79.25,16.0,0.0,5.53,DZ,1.627035e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_name=[]
hotel_df=narrowed_city

location=narrowed_city[["Lat","Lng"]]

basic_url="https://maps.googleapis.com/maps/api/place/textsearch/json"

# API call to find hotels within 5000 meters radius
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params={"key":g_key, "query":"hotel", "location":f"{lat},{lng}", "radius":5000}

    response=requests.get(basic_url,params).json()

    hotel_name.append(response["results"][0]["name"])

# Adding "Hotel Name" column to the DataFrame and displaying it
hotel_df["Hotel Name"]=hotel_name

# Saving a CSV of data file 
hotel_df.to_csv("hotel_output.csv", index=False)

# Displaying Hotel DataFrame
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kamenka,51.3223,42.7678,77.20,33.0,0.0,8.59,RU,1.627034e+09,Gostinyj Dvor Mini-hotel
1,Zlobin,45.2924,14.6541,77.16,30.0,0.0,4.61,HR,1.627035e+09,Uvala Scott
2,Mossamedes,-15.1961,12.1522,75.88,57.0,0.0,5.64,AO,1.627035e+09,Hotel Chik Chik Namibe
3,Lüderitz,-26.6481,15.1594,70.50,20.0,0.0,9.24,NaN,1.627035e+09,LÜDERITZ NEST HOTEL
4,Zarechnyy,53.2036,45.1923,72.48,37.0,0.0,8.57,RU,1.627035e+09,Penza Hotel
5,Henties Bay,-22.1160,14.2845,77.85,20.0,0.0,9.66,NaN,1.627035e+09,Desert Rose
6,Médéa,36.2642,2.7539,79.25,16.0,0.0,5.53,DZ,1.627035e+09,Hôtel Mongorno


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure()


fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(center=(0,0), zoom_level=2)


# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))